In [ ]:
import pandas as pd 
import os
import json
import psycopg2
from sqlalchemy import create_engine

from methods.coinmarketcap_api import get_coins_coinmarketcap

# Get Overview of the coins

* ~Source 1 Coinmarketcap~
* Source 2 CoinGecko
* Source 3 Defilama

In [ ]:
response_not_empty = True
data_list = []
start = 1
while response_not_empty:
    data_json = get_coins_coinmarketcap(start=start)
    data = pd.DataFrame(data_json['data'])
    data_list.append(data)
    start += 5000
    response_not_empty = bool(data_json['data'])
data = pd.concat(data_list)

# Preprocess data

In [ ]:
data['platform'] = data['platform'].apply(lambda x: json.dumps(x))
data['first_historical_data'] = pd.to_datetime(data['first_historical_data'],infer_datetime_format=True)
data['last_historical_data'] = pd.to_datetime(data['last_historical_data'],infer_datetime_format=True)

data['first_historical_data_year'] = data['first_historical_data'].dt.year
data['last_historical_data_year'] = data['last_historical_data'].dt.year

In [ ]:
# init database
# establish connections
# db = create_engine(os.environ["CONN_STRING"])
# conn = db.connect()
# data.to_sql('coins', con=conn, if_exists='replace', index=False)

In [ ]:


df_1 = _deepnote_execute_sql("""SELECT
    MAX(last_historical_data) AS last_update_date
FROM public.coins 
""", 'SQL_7FEBEE12_E87C_46D1_A772_A82F0C235306')
df_1

,last_update_date
0,2022-02-21 18:50:00+00:00


In [ ]:


old_data = _deepnote_execute_sql("""SELECT *
FROM public.coins 
""", 'SQL_7FEBEE12_E87C_46D1_A772_A82F0C235306')
old_data

,id,name,symbol,slug,rank,is_active,first_historical_data,last_historical_data,platform,first_historical_data_year,last_historical_data_year
0,1,Bitcoin,BTC,bitcoin,1.0,1,2013-04-28 18:47:21+00:00,2022-02-20 15:29:00+00:00,null,2013.0,2022.0
1,2,Litecoin,LTC,litecoin,20.0,1,2013-04-28 18:47:22+00:00,2022-02-20 15:29:00+00:00,null,2013.0,2022.0
2,3,Namecoin,NMC,namecoin,646.0,1,2013-04-28 18:47:22+00:00,2022-02-20 15:29:00+00:00,null,2013.0,2022.0
3,4,Terracoin,TRC,terracoin,1853.0,1,2013-04-28 18:47:22+00:00,2022-02-20 15:29:00+00:00,null,2013.0,2022.0
4,5,Peercoin,PPC,peercoin,821.0,1,2013-04-28 18:47:23+00:00,2022-02-20 15:29:00+00:00,null,2013.0,2022.0
...,...,...,...,...,...,...,...,...,...,...,...
26342,18238,Snowy Owl,SNO,snowy-owl,3043.0,1,2022-02-21 12:35:00+00:00,2022-02-21 18:45:00+00:00,"{""id"": 5805, ""name"": ""Avalanche C-Chain"", ""sym...",2022.0,2022.0
26343,18239,Sunny Side up,SSU,sunny-side-up,5537.0,1,2022-02-21 12:30:00+00:00,2022-02-21 18:45:00+00:00,"{""id"": 5426, ""name"": ""Solana"", ""symbol"": ""SOL""...",2022.0,2022.0
26344,18240,Shibrobi,SHIBORG,shibrobi,3919.0,1,2022-02-21 13:17:00+00:00,2022-02-21 18:47:00+00:00,"{""id"": 1027, ""name"": ""Ethereum"", ""symbol"": ""ET...",2022.0,2022.0
26345,18241,DBQuest,DBQ,dbquest,3366.0,1,2022-02-21 13:20:00+00:00,2022-02-21 18:50:00+00:00,"{""id"": 1839, ""name"": ""BNB Smart Chain (BEP20)""...",2022.0,2022.0


In [ ]:
last_update = df_1.iloc[0]['last_update_date']
update_df = data[data['last_historical_data'] > last_update]

In [ ]:
columns_updates = ['id', 'name', 'symbol', 'slug', 'rank', 'is_active', 'platform']

In [ ]:
update_ids = update_df['id'].values

# filter data out that are not includet
data = data[data['id'].isin(update_ids)]

data_to_update = pd.concat([old_data[columns_updates],update_df[columns_updates]]).drop_duplicates(keep=False)

In [ ]:
ids_to_update = data_to_update['id'].unique()

In [ ]:
data = data[data['id'].isin(ids_to_update)]

In [ ]:
db = create_engine(os.environ["CONN_STRING"])
conn = db.connect()
data.to_sql('coins', con=conn, if_exists='append', index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=08dc76ba-15fb-4bf7-936b-bb108060b391' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>